# Amazon Stock Price Direction Prediction

**DISCLAIMER**: This work is for educational purposes only and does not constitute financial advice. The objective of this project is to explore the application of machine learning models in predicting Amazon stock market behavior and to evaluate their limitations.

**What we're doing:** Predicting if tomorrow's Amazon stock will go UP or DOWN

**Why?** If we know the direction, we can:
- Buy at opening if we predict UP
- Don't buy (or sell) if we predict DOWN




**The task:** Binary classification
- Target = 1 means tomorrow closes HIGHER than it opens (UP day)
- Target = 0 means tomorrow closes LOWER than it opens (DOWN day)

**Success criteria:** Get AUC score > 0.515 (better than random guessing)

In [6]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning models
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# Evaluation metrics
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report, confusion_matrix

# Ignore warnings to keep output clean
import warnings
warnings.filterwarnings('ignore')

# Make plots look nice
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("Libraries imported successfully!")

Libraries imported successfully!


We have Amazon stock data from 1997 to 2020 split into three files:
- **Training data** (1997-2016): We use this to teach our model
- **Validation data** (2016-2018): We use this to pick the best model
- **Test data** (2018-2020): We use this for final evaluation (the model has never seen this!)

Each file has the same columns:
- Date
- Open (opening price)
- High (highest price that day)
- Low (lowest price that day)
- Close (closing price)
- Adj Close (adjusted closing price)
- Volume (number of shares traded)

In [7]:
# Load the three CSV files
train_df = pd.read_csv('datasets/AMZN_train.csv', parse_dates=['Date'])
val_df = pd.read_csv('datasets/AMZN_val.csv', parse_dates=['Date'])
test_df = pd.read_csv('datasets/AMZN_test.csv', parse_dates=['Date'])

# Check the sizes
print("Dataset Sizes:")
print(f"  Training:   {len(train_df):,} days")
print(f"  Validation: {len(val_df):,} days")
print(f"  Test:       {len(test_df):,} days")

# Look at the first few rows
print("\nFirst 5 rows of training data:")
train_df.head()

Dataset Sizes:
  Training:   4,781 days
  Validation: 503 days
  Test:       504 days

First 5 rows of training data:


,Date,Open,High,Low,Close,Adj Close,Volume
0,1997-05-15,2.437500,2.500000,1.927083,1.958333,1.958333,72156000
1,1997-05-16,1.968750,1.979167,1.708333,1.729167,1.729167,14700000
2,1997-05-19,1.760417,1.770833,1.625000,1.708333,1.708333,6106800
3,1997-05-20,1.729167,1.750000,1.635417,1.635417,1.635417,5467200
4,1997-05-21,1.635417,1.645833,1.375000,1.427083,1.427083,18853200


Next, we need to create what we're trying to predict. 

**Our target:** Will tomorrow be an UP day or DOWN day?
- UP day (1): Tomorrow's close > tomorrow's open
- DOWN day (0): Tomorrow's close < tomorrow's open

**How we do it:**
1. Shift the data to get "next day" values
2. Compare next day's close to next day's open
3. Create binary target (1 or 0)

In [8]:
# Create target variable
def create_target(df):
    """
    Create the target variable for prediction.
    Returns a dataframe with a new 'target' column.
    """
    # Make a copy so we don't modify the original
    df = df.copy()
    
    # Get tomorrow's open and close by shifting up by 1 row
    # shift(-1) means "give me the next row's value"
    df['next_open'] = df['Open'].shift(-1)
    df['next_close'] = df['Close'].shift(-1)
    
    # Create target: 1 if next close > next open, else 0
    df['target'] = (df['next_close'] > df['next_open']).astype(int)
    
    # Remove the last row (it has no "next day" data)
    df = df[:-1].reset_index(drop=True)
    
    return df

# Apply to all three datasets
train_df = create_target(train_df)
val_df = create_target(val_df)
test_df = create_target(test_df)

# Check target distribution
print("Target distribution in training data:")
print(train_df['target'].value_counts())
print(f"\nPercentage of UP days: {train_df['target'].mean()*100:.1f}%")
print(f"Percentage of DOWN days: {(1-train_df['target'].mean())*100:.1f}%")

Target distribution in training data:
target
1    2392
0    2388
Name: count, dtype: int64

Percentage of UP days: 50.0%
Percentage of DOWN days: 50.0%


### Feature Engineering

This is the most important part! We'll create "features" (clues) that help predict the target.

Think of features as pieces of information the model uses to make decisions.

**We'll create 10 groups of features:**

1. **Basic Price Features** - Today's price movements
2. **Gap Features** - Overnight price changes (very important!)
3. **Moving Averages** - Trend indicators
4. **Volatility Features** - How jumpy is the stock?
5. **Momentum Features** - Is it speeding up or slowing down?
6. **Volume Features** - Trading activity
7. **Pattern Features** - Consecutive up/down days
8. **Technical Indicators** - RSI and MACD (professional tools)
9. **Lag Features** - Yesterday's patterns
10. **Time Features** - Day of week, month, etc.